In [1]:
import sys

if "google.colab" in sys.modules:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Install required packages

    !pip install ase torch_geometric

    import torch
    from torch_geometric.data import Data, DataLoader

    dataset = "/content/drive/My Drive/Dataset"
else:
    dataset = "Dataset"

## Load the data 

In [ ]:
data_torch = torch.load("{dataset}/the_data.pt")

In [ ]:
# Split the data
from sklearn.model_selection import train_test_split

train, the_test = train_test_split(data_torch, test_size=0.2, random_state=42)

val,testing = train_test_split(the_test, test_size=0.5, random_state=42)

In [ ]:
# Create data loader
train_loader = DataLoader(train, batch_size = 32, shuffle=True)
val_loader = DataLoader(val, batch_size = 32, shuffle=False)
test_loader = DataLoader(testing, batch_size = 32, shuffle=False)

In [ ]:
## Model